In [34]:
!pip install pdf2image
!pip install Pillow img2pdf

You should consider upgrading via the 'D:\python310\python.exe -m pip install --upgrade pip' command.


  Created wheel for img2pdf: filename=img2pdf-0.5.1-py3-none-any.whl size=49353 sha256=371ea6214b4d024c9307ed482608aa2632508c9ca9eb54e48912dd07872eabd0
  Stored in directory: c:\users\win 10 pro\appdata\local\pip\cache\wheels\b7\e2\32\e12ea533072415219fc91a248cc413f34c505881ba850bf058
Successfully built img2pdf
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.3.0
    Uninstalling Pillow-9.3.0:
      Successfully uninstalled Pillow-9.3.0


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'D:\\python310\\Lib\\site-packages\\~il\\_imaging.cp310-win_amd64.pyd'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'D:\python310\python.exe -m pip install --upgrade pip' command.


In [37]:
!pip install img2pdf

  Using cached img2pdf-0.5.1-py3-none-any.whl
  Using cached pikepdf-9.4.2-cp310-cp310-win_amd64.whl (3.5 MB)
  Using cached Deprecated-1.2.15-py2.py3-none-any.whl (9.9 kB)


You should consider upgrading via the 'D:\python310\python.exe -m pip install --upgrade pip' command.


In [1]:
from pdf2image import convert_from_path
import numpy as np
from PIL import Image
import cv2
import os
import img2pdf
import shutil


In [3]:
path = 'matrixcookbook.pdf'
poppler_path = r"D:\ml_programs\poppler-24.08.0\Library\bin"
output_path = "temp_book"
landscape_images_path = "processed_book"
desired_dimensions = (2480, 3508)
top = 25
bottom = 10
left = 10
right = 20
added_line_width = 1
line = np.ones((desired_dimensions[0], added_line_width*2, 3), dtype=np.uint8) * np.array((0, 0, 0), dtype=np.uint8)

def book_into_images(path, poppler_path, output_path):
    images = convert_from_path(path, poppler_path=poppler_path, dpi=300)
    for i in range(len(images)):
        # Save pages as images in the pdf
        images[i].save(output_path+'/page'+ str(i) +'.png', 'PNG')

def read_image(image_path):
    image = cv2.imread(image_path)
    return image

def crop_page(image):
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    threshold_value = 128
    _, binary = cv2.threshold(grayscale, threshold_value, 255, cv2.THRESH_BINARY)
    rows_with_content = np.any(binary == 0, axis=1)
    cols_with_content = np.any(binary == 0, axis=0)
    top = np.argmax(rows_with_content)  
    bottom = len(rows_with_content) - np.argmax(np.flipud(rows_with_content)) 
    left = np.argmax(cols_with_content)  
    right = len(cols_with_content) - np.argmax(np.flip(cols_with_content)) 
    cropped_image = image[top:bottom, left:right]
    return cropped_image

def stretch_image(cropped_image, desired_dimensions, top, bottom, left, right, added_line_width):
    width = int(desired_dimensions[1]/2 - (left + right + added_line_width))
    height = int(desired_dimensions[0] - (top + bottom))
    stretched_image = cv2.resize(cropped_image, (width, height))
    return stretched_image

def add_margins(stretched_image, top, bottom, left, right):
    image_with_margins = cv2.copyMakeBorder(
        stretched_image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(255, 255, 255)  # White margins
    )
    return image_with_margins

def concat_image(left_page, right_page, line):
    side_by_side_image = np.hstack((left_page, line, right_page))
    return side_by_side_image

def produce_landscape_page(left_page_path, right_page_path, desired_dimensions, top, bottom, right, added_line_width, line):
    left_page, right_page = read_image(left_page_path), read_image(right_page_path)
    left_cropped_page = crop_page(left_page)
    left_stretched_page = stretch_image(left_cropped_page, desired_dimensions, top, bottom, left, right, added_line_width)
    left_margined_page = add_margins(left_stretched_page, top, bottom, left, right)
                                     
    right_cropped_page = crop_page(right_page)
    right_stretched_page = stretch_image(right_cropped_page, desired_dimensions, top, bottom, left, right, added_line_width)
    right_margined_page = add_margins(right_stretched_page, top, bottom, right, left)

    landscape_page = concat_image(left_margined_page, right_margined_page, line)
    return landscape_page

In [5]:
os.makedirs(output_path)
os.makedirs(landscape_images_path)
book_into_images(path, poppler_path, output_path)

all_image_paths = os.listdir(path=output_path)
all_image_paths.sort(key=lambda x: int(x[4:-4]))
full_image_paths = [output_path+"/"+i for i in all_image_paths]
count = 0
for left_page_path, right_page_path in zip(full_image_paths[::2], full_image_paths[1::2]):
    landscape_page = produce_landscape_page(left_page_path, right_page_path, desired_dimensions, top, bottom, right, added_line_width, line)
    cv2.imwrite(landscape_images_path+"/page"+str(count)+".png", landscape_page)
    count+=1

In [43]:
all_image_paths = os.listdir(path=landscape_images_path)
all_image_paths.sort(key=lambda x: int(x[4:-4]))
full_image_paths = [landscape_images_path+"/"+i for i in all_image_paths]
with open(path[:-4]+"_landscaped.pdf","wb") as f:
	f.write(img2pdf.convert(full_image_paths))



In [2]:
shutil.rmtree('processed_book')
shutil.rmtree("temp_book")